决策树：寻找划分数据集的最好标签
       划分数据集
       创建分支节点
       在每个划分的子集中：寻找最好的标签继续划分数据集

In [110]:
import math
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    featureNames = ['no surfacing','flippers'] #不浮出水面是否存活 ，有无脚蹼
    #change to discrete values
    return dataSet, featureNames

1、对标签列求信息熵
熵：信息的期望值
信息的定义：l(xi) = -log(P(xi))
熵：H(x) = - sum[P(xi)log(P(xi))]



In [111]:
def entropy(dataset):
    num = len(dataset)
    labelSets = {}
    for fec in dataset:
        currentLabel = fec[-1]
        if currentLabel not in labelSets.keys():
            labelSets[currentLabel] = 0
        labelSets[currentLabel] += 1
    
    entropy = 0.0
    for key in labelSets:
        prob = float(labelSets[key]) / num  #P(xi)
        entropy = - prob * math.log2(prob)   #这里不求熵，因为没有求和
#         print('key:{}'.format(key),'value:{}'.format(labelSets[key]),entropy)
    return entropy

熵越高，表明数据越混乱,不确定性就越大

In [112]:
data,fec = createDataSet()
print(data)
my_entropy = entropy(data)
print(my_entropy)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
0.44217935649972373


2、按获取最大信息增益，选取最好的数据集划分数据集

In [113]:
def splitDataSet(dataset,axis,value):
    returnDataset = []
    for i in dataset:
        if i[axis] == value:
            tmp = i[:axis]
            tmp.extend(i[axis+1:]) #1,yes
            returnDataset.append(tmp)
    return returnDataset
        

In [114]:
data,fec = createDataSet()
resu = splitDataSet(data,0,1)  #axis = 0,且这个值为1
print(resu)
print(len(resu))

[[1, 'yes'], [1, 'yes'], [0, 'no']]
3


看哪个特征划分使得信息熵(数据无序度)减小的最多  ---最佳分割特征

In [115]:
def chooseBestFec(dataset):
    numFec = len(dataset[0]) -1  #最后的标签列去除
    baseEntropy = entropy(dataset)  #原始信息熵
    bestInfoGain = 0.0;bestFeature = -1
    for i in range(numFec):
        featureList = [ex[i] for ex in dataset]
        uniqueFec = set(featureList)  #去重
        newEntropy = 0.0
        for value in uniqueFec:
            subData = splitDataSet(dataset,i,value)#对第i个特征，针对某个值划分
#             print('sub:{}'.format(subData))
            prob = len(subData) / float(len(dataset))
            newEntropy += prob * entropy(subData)
        
        infoGain = baseEntropy - newEntropy  #信息增益
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain   #替换好的
            bestFeature = i   
    return bestFeature            

In [116]:
index = chooseBestFec(data)
print(index)

0


In [117]:
'''
找出list中出现次数最多的
对字典排序，取得最大值：将多数的类别标签作为叶子节点的类别
'''
def majority(list):
    return np.argmax(np.bincount(list))

#测试
c = [1,1,1,0,0,2,2,2,2,3,3,3,3,3,3,3,3]
print(majority(c))

3


根据数据集合，创建一个完整的决策树

In [118]:
def createTree(dataSet,featureNames):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): #如果所有的类都一样，第0类的个数==长度
        return classList[0]#stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return majorityCnt(classList) #如果所有的特征都被遍历用于划分数据
    bestFeature = chooseBestFec(dataSet)
    bestFeatureName = featureNames[bestFeature]
    myTree = {bestFeatureName:{}} #用字典存储树结构
    del(featureNames[bestFeature]) #从特征名称列表中删除这个bestFeatureName
    featureValues = [example[bestFeature] for example in dataSet]  #遍历最好的特征的取值，进行划分
    uniqueVals = set(featureValues)
    for value in uniqueVals:
        subNames = featureNames[:]  #copy all of featureName, 为了保留原有的featureName不被函数修改
        myTree[bestFeatureName][value] = \
            createTree(splitDataSet(dataSet, bestFeature, value),subNames) #返回的是一个字典
    return myTree 

In [119]:
myTree = createTree(data,fec)

In [124]:
print(myTree)

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
